## Chroma Quick Start
docs: https://docs.trychroma.com/getting-started


In [ ]:
%pip install chromadb

In [2]:
import chromadb
chroma_client = chromadb.Client()

In [3]:
# create collection
# Collections are where you'll store your embeddings, documents, and any additional metadata. 
# You can create a collection with a name:

collection = chroma_client.create_collection(name='my_collection')

In [4]:
# add some text
# Chroma将存储你的文本，并自动处理标记化、嵌入和索引。

collection.add(
    documents=["This is a document", "This is another document"],
    metadatas=[{"source": "my_source"}, {"source": "my_source"}],
    ids=["id1", "id2"]
)

In [7]:
# 相似度查询
collection.query(query_texts=["This is a document"], n_results=1)

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['This is a document']],
 'metadatas': [[{'source': 'my_source'}]],
 'distances': [[0.0]]}

In [11]:
# collection.add(
#     documents=["王存是个好人", "方朋可真行啊"],
#     metadatas=[{"source": "other"}, {"source": "other"}],
#     ids=["id3", "id4"]
# )
collection.query(query_texts=["谁是好人?"], n_results=1)

{'ids': [['id3']],
 'embeddings': None,
 'documents': [['王存是个好人']],
 'metadatas': [[{'source': 'other'}]],
 'distances': [[0.7229322195053101]]}